In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from stopit import threading_timeoutable as timeoutable
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = "/Users/hamid/piibamm/PyBaMM/GM2022/data/esoh_R/"
oCV_DIR = "/Users/hamid/piibamm/PyBaMM/GM2022/data/ocv/"
fig_DIR = "../figures/figures_fit/"
res_DIR = "../data/results_fit/"
# %matplotlib widget

In [2]:
# parameter_values = get_parameter_values()

parameter_values = pybamm.ParameterValues(chemistry=pybamm.parameter_sets.Andrew2022)

parameter_values.update(
    {
        # mechanical properties
        "Positive electrode Poisson's ratio": 0.3,
        "Positive electrode Young's modulus [Pa]": 375e9,
        "Positive electrode reference concentration for free of deformation [mol.m-3]": 0,
        "Positive electrode partial molar volume [m3.mol-1]": -7.28e-7,
        "Positive electrode volume change": nmc_volume_change_mohtat,
        # Loss of active materials (LAM) model
        "Positive electrode LAM constant exponential term": 2,
        "Positive electrode critical stress [Pa]": 375e6,
        # mechanical properties
        "Negative electrode Poisson's ratio": 0.2,
        "Negative electrode Young's modulus [Pa]": 15e9,
        "Negative electrode reference concentration for free of deformation [mol.m-3]": 0,
        "Negative electrode partial molar volume [m3.mol-1]": 3.1e-6,
        "Negative electrode volume change": graphite_volume_change_mohtat,
        # Loss of active materials (LAM) model
        "Negative electrode LAM constant exponential term": 2,
        "Negative electrode critical stress [Pa]": 60e6,
        # Other
        "Cell thermal expansion coefficient [m.K-1]": 1.48e-6,
        "Lower voltage cut-off [V]": 2.7,
        # Initializing Particle Concentration
        # "Initial concentration in negative electrode [mol.m-3]": x100*parameter_values["Maximum concentration in negative electrode [mol.m-3]"],
        # "Initial concentration in positive electrode [mol.m-3]": y100*parameter_values["Maximum concentration in positive electrode [mol.m-3]"]
    },
    check_already_exists=False,
)

In [3]:
parameter_values.search("diffu")

EC diffusivity [m2.s-1]	2e-18
Electrolyte diffusivity [m2.s-1]	<function electrolyte_diffusivity_Siegel at 0x7f9232081b80>
Inner SEI lithium interstitial diffusivity [m2.s-1]	1e-20
Negative electrode diffusion coefficient [m2.s-1]	8e-14
Negative electrode diffusivity [m2.s-1]	<function graphite_diffusivity_AndrewMPM at 0x7f92320810d0>
Outer SEI solvent diffusivity [m2.s-1]	2.5000000000000002e-22
Positive electrode diffusion coefficient [m2.s-1]	8e-15
Positive electrode diffusivity [m2.s-1]	<function NMC_diffusivity_AndrewMPM at 0x7f9232081700>
Typical lithium ion diffusivity [m2.s-1]	5.34e-10


In [4]:
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        # "loss of active material": ("stress-driven","none"),
        "loss of active material": "stress-driven",
        # "stress-induced diffusion": "true",
        "lithium plating": "irreversible",
    }
)
# spm.print_parameter_info()
param=spm.param

In [5]:
parameter_values.update(
    {
"Electrode width [m]":0.135,
"Nominal cell capacity [A.h]":3.5,
        
# "Maximum concentration in positive electrode [mol.m-3]":37500,
       
# Updating since February 2022        
"Negative electrode porosity":0.3,
"Positive electrode thickness [m]":5.565e-05,
"Negative electrode thickness [m]": 5.5605e-05,
# "Electrode width [m]" : 0.11,      
"Negative particle radius [m]":13.5e-06,
"Positive particle radius [m]":2.1e-06,
"Maximum concentration in negative electrode [mol.m-3]":27200,
"Maximum concentration in positive electrode [mol.m-3]":33700,

    }
)


In [6]:
cell=41

In [7]:
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spm,parameter_values)

Temp=25
dfe.N.iloc[-1]

144

In [8]:
cycles = np.array(dfe['N'].astype('int'))
cycles

array([  0,  39,  66,  93, 144])

In [9]:
HPPC_093 = pd.read_csv("/Users/hamid/Drive Cycle/HPPC_current_cell093_shortened.csv", comment="#", header=None).to_numpy()

pybamm.set_logging_level("NOTICE")


HPPC= pybamm.Experiment(
    [
        ("Run HPPC_093 (A)")
    ] *1,
    termination="50% capacity",
    drive_cycles={"HPPC_093": HPPC_093},
#     cccv_handling="ode",
)



eps_n_data

0.765529259661426

In [10]:
# parameter_values = get_parameter_values()
parameter_values.update(
    {
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
        "Initial temperature [K]": 273.15+Temp,
        "Ambient temperature [K]": 273.15+Temp,
#         "Positive electrode LAM constant proportional term [s-1]": 9.0638e-08,#1.27152e-07
        "Positive electrode LAM constant proportional term [s-1]": 3.43e-07,
#         "Negative electrode LAM constant proportional term [s-1]": 8.7257e-08,#1.27272e-06
        "Negative electrode LAM constant proportional term [s-1]": 1.47e-07,#1.27272e-06

        "Positive electrode LAM constant exponential term": 1.02,
        "Negative electrode LAM constant exponential term": 1.02,
#         "SEI kinetic rate constant [m.s-1]": 6.08e-16, #4.352126e-16, #4.196499e-16, #4.60788219e-16, 1.08494281e-16 ,
        "SEI kinetic rate constant [m.s-1]": 2.76e-16,
#         "EC diffusivity [m2.s-1]": 4.55e-20, #3.6864e-19, #4.56607447e-19,8.30909086e-19,
        "EC diffusivity [m2.s-1]": 1.75e-19,        
#         "SEI growth activation energy [J.mol-1]": 0.0740062164072284, #5997.629, #1.87422275e+04,1.58777981e+04,
        "SEI growth activation energy [J.mol-1]": 0,
#         "Lithium plating kinetic rate constant [m.s-1]": 4.543e-10,
        "Lithium plating kinetic rate constant [m.s-1]": 5.48e-10,
        
        "Initial inner SEI thickness [m]": 0e-09,
        "Initial outer SEI thickness [m]": 5e-09,
        "SEI resistivity [Ohm.m]": 1.3e3,
        
    #Cathode disolution
#         "Positive electrode dissolution exchange current density": 0.0000863673282791135,
        "Positive electrode dissolution exchange current density": 0,
        
        "Negative electrode dissolution exchange current density": 0,
                
        "Positive electrode dissolution nickel SEI coefficient": 0,
        "Negative electrode dissolution nickel SEI coefficient": 0,
        
        "Positive electrode dissolution nickel intercalation coefficient": 0,
        "Negative electrode dissolution nickel intercalation coefficient": 0,
             
        
        
    },
    check_already_exists=False,
)


In [11]:
Temp

25

In [12]:
D_n=[8e-13,8e-14,8e-15,8e-16,8e-17]
D_p=[8e-14,8e-15,8e-16,8e-17,8e-18]
i0_n=[5e-5,4.244e-06,1e-6,5e-7]
i0_p=[5e-5,4.824e-06,1e-6,5e-7]

In [13]:
SOLUTIONS_LIST=[]
for  i in range(len(D_n)):
    parameter_values.update(
    {"Negative electrode diffusion coefficient [m2.s-1]": D_n[i]
    },
         check_already_exists=False,
    )
    sim_long = pybamm.Simulation(spm, experiment=HPPC, parameter_values=parameter_values, 
                            solver=pybamm.CasadiSolver("safe",
                            rtol=1e-6, 
                            atol=1e-6,
                            dt_max=10,                                                      ))
    sol_long= sim_long.solve(initial_soc=1, save_at_cycles=1  )
    SOLUTIONS_LIST.append(sol_long)

2023-07-15 18:07:35.942 - [NOTICE] callbacks.on_cycle_start(174): Cycle 1/1 (14.604 ms elapsed) --------------------
2023-07-15 18:07:35.943 - [NOTICE] callbacks.on_step_start(182): Cycle 1/1, step 1/1: Run HPPC_093 (A)
CasADi - 2023-07-15 18:07:36 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:521]
2023-07-15 18:07:49.808 - [NOTICE] callbacks.on_cycle_end(196): Capacity is now 3.540 Ah (originally 3.540 Ah, will stop at 1.770 Ah)
2023-07-15 18:07:49.808 - [NOTICE] callbacks.on_experiment_end(222): Finish experiment simulati

In [14]:
SOLUTIONS_LIST[0].plot(
    [
#         "Negative particle surface concentration [mol.m-3]",
#,
        "Terminal voltage [V]",

    ]
)

interactive(children=(FloatSlider(value=0.0, description='t', max=16.66638888888889, step=0.1666638888888889),…

#optimization

In [22]:
SOLUTIONS_LIST[2]["Terminal voltage [V]"].entries.shape

(60000,)

In [29]:
voltage_HPPC_data = pd.read_csv("/Users/hamid/piibamm/PyBaMM/GM2022/cycling_aging/HPPC_voltage_cell093.csv");
vvHPPC=voltage_HPPC_data.to_numpy()

In [32]:
vvHPPC.shape

(60000, 1)

In [35]:
aa=SOLUTIONS_LIST[2]["Terminal voltage [V]"].entries - np.transpose(vvHPPC)

In [37]:
aa.shape

(1, 60000)

In [71]:
objective(SOLUTIONS_LIST[2],voltage_HPPC_data)

ValueError: Unable to coerce to Series, length must be 1: given 60000